In [ ]:
import requests
import time
from tqdm import tqdm

In [ ]:
# Test configurations
SERVERLESS_URL = "https://<ngrok-forwarding-url>/function/image-classify" # Replace with your own ngrok forwarding URL
SERVER_URL = "https://<ngrok-forwarding-url>/predict" # Replace with your own ngrok forwarding URL
IMAGE_PATH = "./images/dog.jpg"
NUM_REQUESTS = 150
DURATION = 120  # seconds

In [ ]:
def test_latency(api_url):
    """Measure the latency of the API by sending multiple sequential requests and recording the response times."""
    latencies = []
    for _ in tqdm(range(NUM_REQUESTS), desc="Testing latency"):
        with open(IMAGE_PATH, 'rb') as img:
            start = time.time()
            response = requests.post(api_url, data=img)
            end = time.time()
            latencies.append(end - start)

    avg_latency = sum(latencies) / len(latencies)
    print(f"Average latency: {avg_latency:.4f} seconds")
    return latencies

In [ ]:
def test_throughput(api_url):
    """Test throughput by sending as many requests as possible within a given duration."""
    with open(IMAGE_PATH, 'rb') as img_file:
        img_data = img_file.read()

    request_count = 0
    start_time = time.time()
    end_time = start_time + DURATION

    with tqdm(total=DURATION, desc="Testing throughput", unit='s') as pbar:
        while time.time() < end_time:
            response = requests.post(api_url, data=img_data)
            if response.status_code == 200:
                request_count += 1
            elapsed_time = int(time.time() - start_time)
            pbar.n = elapsed_time
            pbar.refresh()
        pbar.refresh()

    throughput = request_count / DURATION
    print(f"Throughput: {throughput:.2f} requests/second")
    return throughput

In [ ]:
# Test serverless API
print("Testing serverless API...")
serverless_latencies = test_latency(SERVERLESS_URL)
serverless_throughput = test_throughput(SERVERLESS_URL)

In [ ]:
# Test server-based API
print("Testing server-based API...")
server_latencies = test_latency(SERVER_URL)
server_throughput = test_throughput(SERVER_URL)